## Filtering Data

In [1]:
# Install Additional Python Libraries
!pip install -r requirements.txt

In [2]:
from spark_libs import spark_submit
packages = ["com.databricks:spark-csv_2.11:1.5.0"]
spark_submit(packages=packages)

Adding environment variable `PYSPARK_SUBMIT_ARGS`
--packages com.databricks:spark-csv_2.11:1.5.0 pyspark-shell


In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [4]:
# get or create Spark session

app_name = "spark-filtering"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [5]:
url ="https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)

df = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(SparkFiles.get("wine.csv"))
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [6]:
# In case inferSchema doesn't pick up the right types
# columns can be cast to different data types
df = df \
    .withColumn("points", F.col("points").cast("double")) \
    .withColumn("price", F.col("price").cast("double"))
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: double (nullable = true)
 |-- price: double (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [7]:
df.head()

Row(country='US', description='This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022���2030.', designation="Martha's Vineyard", points=96.0, price=235.0, province='California', region_1='Napa Valley', region_2='Napa', variety='Cabernet Sauvignon', winery='Heitz')

In [8]:
# Order a DataFrame by ascending values

# df.orderBy(df["points"].asc()).show(5)

df.orderBy(F.col("points").asc()).show(5)

+---------+--------------------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|            province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|       US|"The flagship of ...| chewier finish t...|  null| null|             tobacco| cedar and a mult...| starting in 2010...|Limited Release I...|                  93|
|       US|"Note the ""Bened...| from a time when...|  null| null|          sweet beet|          smoky char|          spice. Dry|         full-bodied| rich in dark che...|
|       US|"I called the '04...| and this wine se...|  null| null|  Clos Pepe Vineyard|                  82|                  40|          C

In [9]:
df.select(F.avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [10]:
df.filter("price<20").show()

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
| Bulgaria|This Bulgarian Ma...|             Bergul̩|  90.0| 15.0|        Bulgaria|                null|                null|              Mavrud|        Villa Melnik|
|    Spain|Earthy plum and c...|              Amandi|  90.0| 17.0|         Galicia|       Ribeira Sacra|                null|             Menc�_a|      Don Bernardino|
|       US|There's a lot to ...|                null|  90.0| 18.0|      California|Russian River Valley|              Sonoma|          Chardonnay|            De

In [11]:
# Filter by price on certain columns
df.filter("price<20").select(['points','country', 'winery','price']).show()

+------+---------+--------------------+-----+
|points|  country|              winery|price|
+------+---------+--------------------+-----+
|  90.0| Bulgaria|        Villa Melnik| 15.0|
|  90.0|    Spain|      Don Bernardino| 17.0|
|  90.0|       US|            De Loach| 18.0|
|  91.0|       US|   Trinity Vineyards| 19.0|
|  91.0| Portugal|Adega Cooperativa...| 15.0|
|  86.0|       US|      Belle Ambiance| 10.0|
|  86.0| Portugal| Adega de Cantanhede| 12.0|
|  86.0|       US|            Parducci| 13.0|
|  86.0| Portugal|    Quinta do Portal| 10.0|
|  86.0|   France|               Rigal| 14.0|
|  86.0|       US|     The Naked Grape| 18.0|
|  86.0|   France|   Georges Vigouroux| 15.0|
|  86.0|   France|   Georges Vigouroux| 10.0|
|  86.0|       US| Martinez & Martinez| 17.0|
|  86.0|       US|           Ironstone| 12.0|
|  86.0|       US|       Leaping Horse| 10.0|
|  86.0|       US|        Kitchen Sink| 13.0|
|  86.0| Portugal|  Wines & Winemakers| 12.0|
|  86.0|Argentina|              Zo

### Using Python Comparison Operators

In [12]:
# Same results only this time using python
df.filter(df["price"] < 200).show()

+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|          region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|  96.0|110.0|    Northern Spain|              Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|  96.0| 90.0|        California|    Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|  96.0| 65.0|            Oregon| Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
| France|This is the top w...|    

In [13]:
df.filter( (df["price"] < 200) | (df['points'] > 80) ).show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|  96.0|235.0|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|  96.0|110.0|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|  96.0| 90.0|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [14]:
df.filter(df["country"] == "US").show()

+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|  96.0|235.0|California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|  96.0| 90.0|California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|  96.0| 65.0|    Oregon|   Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
|     US|This re-named vin...|              Silice|  95.0| 65.0|    Or